## Groupe : MAZOUZ Abderahim , FLEURY Manon

In [ ]:
! pip install sentence_transformers rank_bm25 nltk

# Objectives

In this lab work, you are going to elabotrate a simple semanti search algorithm as it is one of the building block of RAGs.

# Instructions
- Have a tidy straightforward code
- Readability over performances
- Define Functions and comments
- Read the doc
- Send me your submission with the names of the students of your group by next week (18/11)!

# Imports and data

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import resample

from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Data

Upload the data to collab and download the csv file.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_qq = pd.read_csv("/content/drive/MyDrive/Copie de qora_qestions.csv") # Change the name of the file if necessary
df_qq.head(10)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


## Initial data analysis

Perform some initial data analysis

In [ ]:
df_qq['is_duplicate'].value_counts()

,count
is_duplicate,
0,255027
1,149263


# Retriever Implementation and Evaluation

## Evaluation method

The dataset is composed of sets of pairs of Qora questionsz with a label indicating if they are to be considered duplicates or not.
An efficient retriever shall retrieve the relevant question out of all the questions.
The retreivers should be evaluated using **at least the Hit Rate and possibly the MAPS.**

## Data structure

In order to evaluate the retrieving methods, you should create the fol:lowing data sets:
1. A base of unique question with their corresponding ``qid``
2. A key-value dataframe matching ``qid``pairs to a ``is_duplicate`` label

Considering, the number of unique questions (ca 55000) you may subsample the dataset to a manageable size.

## Data preparation code

### Data structures creation

In [ ]:
df_1 = df_qq[['qid1','question1']].rename(columns={'qid1':'qid','question1':'question'})
df_2 = df_qq[['qid2','question2']].rename(columns={'qid2':'qid','question2':'question'})
df_q = pd.concat([df_1,df_2],axis=0).drop_duplicates()

In [ ]:
df_labels = df_qq[['qid1','qid2','is_duplicate']]


### Data subsampling

Subsample the data so that you have a smaller dataset.
1. Select a set of questions
2. Select the queries as questions from the previous set having at least 1 positive duplicate
3. **Be sure that the ``qid`` of the selected questions are not part of the set of questions that will be indexed in the k-NN for the retriever**

In [ ]:
potential_queries_indices = resample(df_labels.loc[df_labels['is_duplicate'] == 1, 'qid1'].unique(),replace=False, n_samples=1000)
correct_outputs = df_labels.loc[(df_labels['is_duplicate'] == 1)&(df_labels['qid1'].isin(potential_queries_indices)), 'qid2'].unique()
true_positive_basis = set(correct_outputs) - set(potential_queries_indices)
false_examples = resample(df_q.loc[~df_q['qid'].isin(true_positive_basis), 'qid'].values,
                          n_samples=10000,
                          replace=False)

all_ids = np.concatenate((list(true_positive_basis), false_examples)) # ids of the chunks for the vdb

questions_with_duplicates = df_q.loc[df_q['qid'].isin(potential_queries_indices)] # -> queries for the retriever
df_chunks = df_q.loc[df_q['qid'].isin(all_ids)] # -> chunks to be indexed

### Evaluation functions

Define the relevant functions :
1. match index from questions to labels, by default, if the index pair iis not in the labelling table, consider that the the pair is not duplicates.
2. Compute the metrics
3. Evaluation loop over the relevant questions

In [ ]:
def hits(list_labels):
    """
    Check if a list contains at least one relevant item.

    Args:
        list_labels (list): A list of binary values (1 for relevant, 0 for irrelevant).

    Returns:
        int: 1 if at least one relevant item is found, otherwise 0.
    """
    return int(1 in list_labels)


In [ ]:
def ap(list_labels):
    """
    Compute Average Precision (AP) for a ranked list of binary labels.

    Args:
        list_labels (list): A list of binary values (1 for relevant, 0 for irrelevant).

    Returns:
        float: Average Precision (AP) score.
    """
    # Initialize counters
    relevant_count = 0  # Number of relevant items encountered
    precisions = []     # List to store precision values at each relevant item

    # Iterate through the ranked list
    for i, label in enumerate(list_labels):
        if label == 1:  # Check if the current item is relevant
            relevant_count += 1
            precision_at_i = relevant_count / (i + 1)  # Compute precision at this index
            precisions.append(precision_at_i)

    # Handle case where there are no relevant items
    if relevant_count == 0:
        return 0.0

    # Compute and return the Average Precision
    return sum(precisions) / relevant_count


In [ ]:
def compute_metrics(list_list_labels):
  hits_list = [hits(l) for l in list_list_labels]
  ap_list = [ap(l) for l in list_list_labels]
  return {'hit_rate': sum(hits_list)/len(hits_list), 'map': sum(ap_list)/len(ap_list)}

In [ ]:
test_output = [1,0,0,1,1,0]

print(hits(test_output))#expects 1
print(ap(test_output)) #expects 0.7

1
0.7000000000000001


In [ ]:
list_list_labels = [[1,0,0,1,1,0],[0,0,0,0,0,0]]
compute_metrics(list_list_labels)

{'hit_rate': 0.5, 'map': 0.35000000000000003}

## Retriever implementation

### First Pipeline

In this first Pipeline we are going to use a simple sklearn pipeline for seamntic search.
#### Steps :
- TfIdf for vectorization
- [NearestNeighbors](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors) class for the search engine

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

chunks = df_chunks['question'].to_list()

tfidf_model = TfidfVectorizer() # Initialize the TfidfVectorizer
chunks_matrix = tfidf_model.fit_transform(chunks) # Fit and transform the documents to TF-IDF feature matrix

## knn
vdb_tfidf = NearestNeighbors(n_neighbors=5, metric='cosine')
vdb_tfidf.fit(chunks_matrix)

NearestNeighbors(metric='cosine')

In [ ]:
def predict_duplicates_tfidf(query):
  # You can change it if you want
  query_vec = tfidf_model.transform([query])
  distances, indices = vdb_tfidf.kneighbors(query_vec)
  return df_chunks.iloc[indices[0]]

In [ ]:
qq = questions_with_duplicates.iloc[100]
print('query : \n id: {} question: {}'.format(qq.qid, qq.question))
duplicate_q = df_chunks[df_chunks['qid']==df_labels.loc[(df_labels['is_duplicate']==1) & (df_labels['qid1']==qq.qid), 'qid2'].iloc[0]].iloc[0]
print('duplicate : \n id: {} question: {}'.format(duplicate_q.qid, duplicate_q.question))
predict_duplicates_tfidf(qq['question'])

query : 
 id: 44369 question: Do your parents understand you?
duplicate : 
 id: 203927 question: Do you think your parents understand you? Why or why not?


,qid,question
126563,203927,Do you think your parents understand you? Why ...
370257,500800,What do you do when your parents don't believe...
231601,341451,What don't your parents know about you?
135124,215853,What do you understand by the term “experience”?
131241,210468,What is the worst thing your parents have ever...


#### Evaluation loop

Sample of code, you can change it if you want.

In [ ]:
def get_label(question_id, candidate_id, df_labels):
  a = df_labels.query("qid1 == @question_id and qid2 == @candidate_id")['is_duplicate']
  if a.empty:
    return 0
  return a.iloc[0]

In [ ]:
all_labels = []

for id, question in tqdm(questions_with_duplicates.iloc[:500].iterrows()):
  query = question['question']
  results = predict_duplicates_tfidf(query)
  labels = [get_label(question['qid'], r['qid'], df_labels) for _, r in results.iterrows()]
  all_labels.append(labels)

compute_metrics(all_labels)

0it [00:00, ?it/s]

{'hit_rate': 0.976, 'map': 0.9088638888888887}

### Second Pipeline : Sentence transformer pipeline

#### Steps:
 - Use the sentence_transformers package to load a model and embed the reviews
 - Since it is much more demanding than TfIdf, it could be necessary to subsample the dataset randomly
 - Build a pipeline with the same NearestNeighbors engine.

In [ ]:
!pip install -U datasets

In [ ]:
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
from datasets import Dataset
ds = Dataset.from_dict({'sentences': chunks})
ds = ds.map(lambda x: {'sentence_embeddings': sbert_model.encode(x["sentences"])})

Map:   0%|          | 0/11672 [00:00<?, ? examples/s]

In [ ]:
vdb_sbert = NearestNeighbors(n_neighbors=5, metric='cosine')
vdb_sbert.fit(ds['sentence_embeddings'])

NearestNeighbors(metric='cosine')

In [ ]:
def predict_duplicates_sbert(query):
    #You may change this code
    query_vec = sbert_model.encode([query])
    distances, indices = vdb_sbert.kneighbors(query_vec)
    return df_chunks.iloc[indices[0]]

In [ ]:
all_labels = []

for id, question in tqdm(questions_with_duplicates.iloc[:500].iterrows()):
  query = question['question']
  results = predict_duplicates_sbert(query)
  labels = [get_label(question['qid'], r['qid'], df_labels) for _, r in results.iterrows()]
  all_labels.append(labels)

compute_metrics(all_labels)

0it [00:00, ?it/s]

{'hit_rate': 0.994, 'map': 0.9502583333333332}

### Third Pipleine: BM25

After using distance based searchs methodfs, we are going to work with the BM25 algorithm. It was developped in late 70s but it is still very relevant. Some preprocessing of the text will be needed. A classical pipeline will be implemented.
#### Preprocessing

##### Stopwords

In [ ]:
stopwords = stopwords.words('english')

##### Stemming

In [ ]:
stemmer = SnowballStemmer('english')

#### BM25 engine

In [ ]:
# Create the BM25 Okapi engine
!pip install -q rank-bm25

In [ ]:
tokenized_corpus = [chunk.split(" ") for chunk in chunks]
bm25_model = BM25Okapi(tokenized_corpus)

### Queries

#### Preprocessing

In [ ]:
def query_preprocessing(query):
  # Preprocessing function for the different queries
  words = query.split()
  processed_query = [stemmer.stem(word) for word in words if word not in stopwords]
  return processed_query

In [ ]:
query = questions_with_duplicates.iloc[0]['question']
preprocessed_query = query_preprocessing(query)

print(query)
print(preprocessed_query)
best_matches = bm25_model.get_top_n(preprocessed_query, df_chunks.index, n=5)
df_chunks.loc[best_matches]

What is the difference between the iPhone 6s and iPhone 6s Plus?
['what', 'differ', 'iphon', '6s', 'iphon', '6s', 'plus?']


,qid,question
169133,33005,"Aside from screen size, what is different betw..."
111206,182206,What is the difference between iPhone 6s and i...
95846,159764,Should I buy the iPhone 6s or an SE?
348618,477198,Can I make calls with I phone 6s or any app re...
29377,54358,How use Miracast screen mirroring by Moto g4 p...


In [ ]:
def predict_duplicates_bm25(query):
    # You may change that
    preprocessed_query = query_preprocessing(query)
    best_matches = bm25_model.get_top_n(preprocessed_query, df_chunks.index, n=5)
    return df_chunks.loc[best_matches]

In [ ]:
all_labels = []

for id, question in tqdm(questions_with_duplicates.iloc[:500].iterrows()):
  query = question['question']
  results = predict_duplicates_bm25(query)
  labels = [get_label(question['qid'], r['qid'], df_labels) for _, r in results.iterrows()]
  all_labels.append(labels)

compute_metrics(all_labels)

0it [00:00, ?it/s]

{'hit_rate': 0.686, 'map': 0.5752721428571428}

# Conclusion

Explain the results:
- how does the different methods compare ?
- What can be an explaination for the difference between the results of BM25 and TF-IDF veectors in a kNN?


# If you have more time (not evaluated):

If you have some spare time, you can:
1. Try to download and use a cross-encoder model
2. Try to use a HNSW k-NN from the FAISS library

**Explanation of the results**

Results:

1. TfIdf for vectorization and KNN: {'hit_rate': 0.98, 'map': 0.91}

2. Sentence transformer: {'hit_rate': 0.998, 'map': 0.96}

3. BM25: {'hit_rate': 0.64, 'map': 0.53}

We see that Sentence transformer performs the best, and BM25 the worst.

Sentence Transformers are designed to capture the semantic meaning of sentences, enabling them to find duplicates even when the words differ. This is why they perform best in detecting duplicates that are paraphrased or reworded.

TfIdf +KNN works well when the questions share the same terminology, but its reliance on keyword matching limits its ability to detect duplicates that are semantically similar but not lexically identical.

BM25, while very good for classic information retrieval tasks where exact or close word matches are important, is less effective when semantic similarity is required. It doesn't capture the deeper context of words, which results in lower performance for detecting duplicates.

> Comparison TfIdf-BM25:

**BM25** is optimized for keyword-based search, ranking documents based on term frequency and document frequency, but it doesn't capture semantic meaning (because it relies on exact term matching), which limits its ability to detect paraphrased or reworded questions. In contrast, **TFIDF +KNN** relies on vector representations of text and measures the cosine similarity between these vectors, which works well when there is word overlap. TF-IDF vectors can provide better matches when questions have similar terminology, while BM25 struggles when the phrasing differs, even if the underlying meaning is the same.